In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from fastai.vision.all import *
from tqdm.notebook import tqdm
from multiprocessing import Pool
from PIL import Image

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
picked_labels = ["Atelectasis"]#["Atelectasis", "Cardiomegaly", "Pneumonia"]
p_dev_images = get_data_path()/"dev_images"

In [3]:
def namify(label):
    return "".join(label.split(" ")).lower()

def get_label_func(label):
    return partial(partial_label_func, label)

def partial_label_func(label, fname):
    lbl = complete_df[complete_df["Image Index"] == fname.name][label].values[0]
    return "Pos" if lbl > 0 else "Neg"
#model_name = f"xresnet50_finetuned_{namify(label)}_v1"


architecture = "xresnet50_finetuned"

In [4]:
label = picked_labels[0]
s = 224
image_size = (s, s)
labels = get_labels()
bs = 16
device = get_device()
model_name = f"{architecture}_{namify(label)}_v1"

Using the GPU!


In [ ]:
train_df, valid_df, test_df = get_dataframes(include_labels=get_labels(), 
                                                 small=False)
#print(train_df.shape, valid_df.shape, test_df.shape)
train_df = get_binary_df(label, train_df)
valid_df = get_binary_df(label, valid_df)
test_df  = get_binary_df(label, test_df)

complete_df = pd.concat([train_df, valid_df, test_df])
complete_df = complete_df.drop(columns=["Follow-up #", "Patient ID", "Patient Age", "Patient Gender", 
                   "View Position", "OriginalImage[Width", "Height]", 
                   "OriginalImagePixelSpacing[x", "y]", "No Finding"])
#train_df, valid_df, complete_df

In [ ]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y     = get_label_func(label),
                   splitter  = RandomSplitter(),
                   item_tfms = [Resize(224, 
                                       method="squish", 
                                       pad_mode="zeros")])
dls = dblock.dataloaders(p_dev_images, bs=bs)

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, 
                model, 
                loss_func=LabelSmoothingCrossEntropy(),
                metrics=[accuracy, RocAucBinary()],
                cbs=callbacks)

In [5]:
model = xresnet50(n_out=2)

#model = load_model(Path().cwd()/"models"/f"{model_name}.pth",
#                   xresnet50(),
#                   LabelSmoothingCrossEntropy(),
#                   with_opt=False, device=device)
model.load_state_dict(torch.load(Path().cwd()/"models"/f"{model_name}.pth")["model"])
#model.eval();
#model = model.to(device)

<All keys matched successfully>

In [6]:
model.eval();
model = model.to(device)

In [7]:
train_df, _, test_df = get_dataframes(include_labels=labels, 
                                             small=False)

train_df = get_binary_df(label, train_df)
test_df = get_binary_df(label, test_df)

In [8]:
_, test_tfs = get_transforms(image_size=image_size)
test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)
test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
criterion = LabelSmoothingCrossEntropy()

In [9]:
dl_iter = test_dl._get_iterator()
X, y = next(dl_iter)
while y.sum() == 0:
    X, y = next(dl_iter)
X, y = X.to(device), y.to(device)

In [ ]:
nn.Softmax()(model(X)), y

In [10]:
y_pred = nn.Softmax()(model(X))

<ipython-input-10-5849c321ab69>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_pred = nn.Softmax()(model(X))


In [18]:
y_pred.max(dim=1).values

tensor([0.9049, 0.9298, 0.9237, 0.9229, 0.9326, 0.9173, 0.9228, 0.9373, 0.9295,
        0.9293, 0.9211, 0.9277, 0.9264, 0.9104, 0.9342, 0.9244],
       device='cuda:0', grad_fn=<MaxBackward0>)

In [ ]:
history = validate(model, 
             criterion, 
             test_dl, 
             model_name,
             device=device)

In [ ]:
#for label in picked_labels:
for label in ["Atelectasis"]:
    print(f"Computing metrics for '{label}'")
    
    seed = 92
    model_name = f"{architecture}_{namify(label)}_v1"
    print(model_name)
    s = 224
    image_size = (s, s)
    labels = get_labels()
    bs = 16
    device = get_device()
    
    sigmoid = NN.Sigmoid()
    model = load_model(model_name)
    model.eval();
    model = model.to(device)
    
    seed_everything(seed=seed)
    
    train_df, _, test_df = get_dataframes(include_labels=labels, 
                                             small=False)

    train_df = get_binary_df(label, train_df)
    test_df = get_binary_df(label, test_df)

    train_label = train_df[[label]].values
    neg_weights, pos_weights = compute_class_freqs(train_label)
    neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)

    _, test_tfs = get_transforms(image_size=image_size)

    test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)

    test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)

    criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                              neg_weights.to(device))

    history = validate(model, 
             criterion, 
             test_dl, 
             model_name,
             device=device)
    
    tmp = history
    tmp["loss"] = tmp["loss"].mean()
    tmp = {k:[v] for k, v in tmp.items()}
    model_metrics = pd.DataFrame(tmp.values(), index=tmp.keys(), columns=[model_name])
    #save_metrics(model_metrics, model_name)
    
FERTIG()

In [ ]:
model_metrics